In [1]:
# ===== Cell 1: 로깅 설정 =====
import logging

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S',
    force=True  # ← 중요!
)

logger = logging.getLogger(__name__)
logger.info("✅ 로깅 시스템 초기화 완료")
import os
import json
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
import pyautogui
from config import NAVERBAND_LOGIN_URL

[00:06:57] [INFO] ✅ 로깅 시스템 초기화 완료


In [2]:
from src.chrome_manager import select_mobile_and_get_driver
mobile, driver = select_mobile_and_get_driver()
from src.utils import *
from src.naverband_automation import *

[00:07:05] [INFO] 📱 선택한 전화번호: 01027851965
[00:07:05] [INFO] ✅ 사용할 User-Agent: Chrome 142
[00:07:05] [INFO] get_cache_timestamp_file()->cache_timestamp_path의 경로는 d:\coding\band_project\accounts/01027851965/.chromedriver_cache_timestamp
[00:07:05] [INFO] get_profile_paht()->profile_dir의 경로는 d:\coding\band_project\accounts/01027851965/chrome_profile
[00:07:05] [INFO] ✅ Preferences 수정 완료: d:\coding\band_project\accounts/01027851965/chrome_profile
[00:07:05] [INFO] 🚀 드라이버 생성 중... (Chrome 142)
[00:07:09] [INFO] patching driver executable C:\Users\jknam\appdata\roaming\undetected_chromedriver\undetected_chromedriver.exe
[00:07:14] [INFO] ✅ User-Agent 강제 적용 완료 (CDP)
[00:07:15] [INFO] ✅ selenium-stealth 적용 완료
[00:07:15] [INFO] ✅ 드라이버 생성 완료
[00:07:19] [INFO] ✅ [01027851965] 드라이버 준비 완료


In [3]:
from src.account_manager import login
login(driver)

[00:07:27] [INFO] [INFO] 쿠키 추가 완료: 성공 5개, 실패 0개
[00:07:27] [INFO] [INFO] 쿠키 복원 완료: d:\coding\band_project\accounts/01027851965/cookies\cookies.json
[00:07:27] [INFO] [INFO] 쿠키 기반 자동 로그인을 시도합니다.
[00:07:34] [INFO] [INFO] 전화번호 입력 성공
[00:07:37] [INFO] [INFO] 비밀번호 입력 성공
[00:07:40] [INFO] [INFO] 2차 인증 없음, 로그인 완료
[00:07:40] [INFO] [INFO] 로그인 완료, 쿠키 저장은 외부에서 수행 예정


True

In [13]:
def process(driver, xpath_dict, band, TXT_DIR, IMAGE_DIR):
    
    focus_window("band.us | band") # 창 포커스 맞추기가 문제의 핵심이었음
    move_mouse_naturally()
    x_path_click(driver, band)
    human_delay("click")

    human_delay("thinking")
    x_path_human_click(driver, xpath_dict['글쓰기_1'])

    write_text_from_folder(driver, xpath_dict['글쓰기_2'], TXT_DIR)
    human_delay("typing")

    upload_file_from_folder(driver, IMAGE_DIR)
    human_delay("upload")

    # move_mouse_naturally()
    x_path_click(driver, xpath_dict['이미지첨부'])
    human_delay("click")

    x_path_click(driver, xpath_dict['이미지게시'])
    human_delay("thinking")

    driver.execute_script("window.scrollBy(0, window.innerHeight / 3)")
    human_delay("scroll")
    x_path_click(driver, xpath_dict['홈'])
    human_delay("scroll")


In [ ]:
def safe_go_home(driver):
    """홈 복귀 + 밴드 목록 로드 확인"""
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    
    try:
        # 홈 URL로 이동
        driver.get(DEFAULT_HOME_URL)
        logger.info(f"홈 URL로 이동: {DEFAULT_HOME_URL}")
        
        # 밴드 목록 로드 대기 (최대 10초)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="content"]/section/div[2]/div/ul')
            )
        )
        
        time.sleep(1)  # 안정화
        logger.info("✅ 홈 복귀 완료")
        return True
        
    except Exception as e:
        logger.error(f"홈 복귀 실패: {e}")
        return False

In [16]:
def go_home(driver):
    """홈 복귀 + 밴드 목록 로드 확인"""
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    
    try:
        # 홈 URL로 이동
        driver.get(DEFAULT_HOME_URL)
        logger.info(f"홈 URL로 이동: {DEFAULT_HOME_URL}")
        
        # 밴드 목록 로드 대기 (최대 10초)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="content"]/section/div[2]/div/ul')
            )
        )
        
        time.sleep(1)  # 안정화
        logger.info("✅ 홈 복귀 완료")
        return True
        
    except Exception as e:
        logger.error(f"홈 복귀 실패: {e}")
        return False

In [17]:
def roof_bands(driver, xpath_dict, BAND_LIST, TXT_DIR, IMAGE_DIR, mobile_num=None):
    """
    밴드 순회 함수
    
    Args:
        driver: Selenium WebDriver
        xpath_dict: XPath 딕셔너리
        BAND_LIST: 밴드 목록
        TXT_DIR: 텍스트 디렉토리
        IMAGE_DIR: 이미지 디렉토리
        mobile_num (str, optional): 계정 번호 (로깅용)
    
    Returns:
        list: 실패한 밴드 목록
    """
    # ===== 초기화 =====
    band_list = BAND_LIST.copy()
    random.shuffle(band_list)
    
    total_bands = len(band_list)
    success_count = 0
    failed_bands = []
    
    # 로깅용 프리픽스
    prefix = f"[{mobile_num}] " if mobile_num else ""
    logger.info(f"{prefix}🚀 밴드 순회 시작 (총 {total_bands}개)")
    
    # ===== 밴드 순회 =====
    while band_list:
        band = band_list[0]
        current = total_bands - len(band_list) + 1
        band_success = False
        
        try:
            logger.info(f"{prefix}[{current}/{total_bands}] 밴드 처리 시작")
            
            # 밴드 처리
            process_band(driver, xpath_dict, band, TXT_DIR, IMAGE_DIR)
            
            # 성공!
            band_success = True
            logger.info(f"{prefix}[{current}] ✅ 성공!")
        
        # ===== 예외 처리 =====
        except UnexpectedAlertPresentException:
            # Alert 팝업 처리
            logger.warning(f"{prefix}[{current}] ⚠️  Alert 팝업 감지")
            
            # ✅ 당신의 함수 사용!
            handle_js_alert(driver, action='accept')
            human_delay("thinking")
            
            # Alert 처리 후 홈 복귀
            try:
                go_home(driver)
            except Exception as home_err:
                logger.error(f"{prefix}홈 복귀 실패: {home_err}")
        
        except Exception as e:
            # 그 외 모든 에러
            error_type = type(e).__name__
            error_msg = str(e)
            logger.warning(f"{prefix}[{current}] ⚠️  에러 ({error_type}): {error_msg[:100]}")
            
            # ✅ 여기서도 Alert 처리 (당신의 함수)
            handle_js_alert(driver, action='accept')
            
            # 홈 복귀 시도
            try:
                go_home(driver)
            except Exception as home_err:
                logger.error(f"{prefix}홈 복귀 실패: {home_err}")
        
        # ===== 후처리 =====
        finally:
            # 성공 여부에 따라 처리
            if band_success:
                band_list.remove(band)
                success_count += 1
            else:
                band_list.remove(band)
                failed_bands.append(band)
            
            # 다음 밴드 전 대기 및 마우스 이동
            if band_list:
                sleep_time = random.randint(5, 15)
                logger.info(f"{prefix}[{current}] 완료, 남은 밴드: {len(band_list)}개, {sleep_time}초 대기")
                time.sleep(sleep_time)
                move_mouse_naturally()
    
    # ===== 결과 요약 =====
    logger.info("=" * 50)
    logger.info(f"{prefix}🎉 밴드 순회 완료!")
    logger.info(f"{prefix}✅ 성공: {success_count}/{total_bands}개")
    logger.info(f"{prefix}❌ 실패: {len(failed_bands)}/{total_bands}개")
    logger.info("=" * 50)
    
    if failed_bands:
        logger.warning(f"{prefix}실패한 밴드 목록: {failed_bands}")
    
    return failed_bands


In [ ]:
roof_bands(driver, xpath_dict, BAND_LIST, TXT_DIR, IMAGE_DIR, mobile_num=None)

In [ ]:
# main.py

import time
import random
import logging
import pyautogui
from selenium.common.exceptions import InvalidSessionIdException, NoSuchWindowException

from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login, perform_logout, save_cookies
from src.naverband_automation import roof_bands, handle_js_alert, safe_go_home
from src.utils import move_mouse_naturally, human_delay
from resources.xpath_dict import xpath_dict
from config import BAND_LIST, TXT_DIR, IMAGE_DIR, DEFAULT_HOME_URL

logger = logging.getLogger(__name__)


def main():
    """메인 실행 함수"""
    driver = None
    mobile = None
    
    try:
        logger.info("=" * 60)
        logger.info("🚀 프로그램 시작")
        logger.info("=" * 60)
        
        # ===== 1. 드라이버 초기화 =====
        mobile, driver = select_mobile_and_get_driver()
        logger.info(f"✅ {mobile} 계정으로 브라우저 실행됨")
        
        ===== 2. 로그인 =====
        login(driver)
        logger.info("✅ 로그인 완료")
        
        # ===== 3. 실행 시간 입력 (입력 검증) =====
        while True:
            p_time = pyautogui.prompt(
                title="실행시간",
                default='1',
                text="시간을 입력하세요 (예: 1 = 1시간, 0.5 = 30분)"
            )
            
            # 취소 버튼 처리
            if p_time is None:
                logger.warning("⚠️  사용자가 취소를 눌렀습니다")
                return
            
            # 입력 검증
            try:
                p_time_float = float(p_time)
                if p_time_float <= 0:
                    pyautogui.alert("0보다 큰 숫자를 입력하세요!", "입력 오류")
                    continue
                break
            except ValueError:
                pyautogui.alert("숫자만 입력 가능합니다!", "입력 오류")
                continue
        
        # ===== 4. 시간 계산 =====
        e_time = time.time() + 60 * 60 * p_time_float
        
        start_time = time.strftime('%p %I시%M분%S초', time.localtime())
        end_time = time.strftime('%p %I시%M분%S초', time.localtime(e_time))
        
        logger.info("=" * 60)
        logger.info(f"📱 계정: {mobile}")
        logger.info(f"🕐 시작 시간: {start_time}")
        logger.info(f"🕐 종료 예정: {end_time}")
        logger.info(f"⏱️  실행 시간: {p_time_float}시간")
        logger.info("=" * 60)
        
        # ===== 5. 밴드 순회 반복 =====
        rotation_count = 0
        
        while time.time() < e_time:
            rotation_count += 1
            
            logger.info("=" * 60)
            logger.info(f"🔄 {rotation_count}회차 밴드 순회 시작")
            logger.info("=" * 60)
            
            # 밴드 순회 실행
            failed_bands = roof_bands(
                driver=driver,
                xpath_dict=xpath_dict,
                BAND_LIST=BAND_LIST,
                TXT_DIR=TXT_DIR,
                IMAGE_DIR=IMAGE_DIR,
                mobile_num=mobile
            )
            
            logger.info(f"✅ {rotation_count}순환 완료 (실패: {len(failed_bands)}개)")
            
            # 남은 시간 확인
            if time.time() < e_time:
                # 다음 회차 전 휴식
                rand_sleep = random.randint(10, 20)
                move_mouse_naturally()
                
                restart_time = time.strftime(
                    '%p %I시%M분',
                    time.localtime(time.time() + rand_sleep)
                )
                
                logger.info("-" * 60)
                logger.info(f"😴 {rotation_count+1}회차 시작, {rand_sleep}초간 휴식")
                logger.info(f"🔄 {restart_time}에 다시 시작합니다")
                logger.info("-" * 60)
                
                time.sleep(rand_sleep)
        
        # ===== 6. 정상 종료 =====
        logger.info("=" * 60)
        logger.info(f"🎉 예정된 시간 종료!")
        logger.info(f"📊 총 {rotation_count}회 순회 완료")
        logger.info("=" * 60)
    
    except KeyboardInterrupt:
        logger.warning("⚠️  사용자가 프로그램을 중단했습니다 (Ctrl+C)")
    
    except Exception as e:
        logger.error(f"❌ 프로그램 실행 중 예외 발생: {e}", exc_info=True)
    
    finally:
        # ===== 7. 종료 처리 =====
        if driver is not None:
            try:
                logger.info("=" * 60)
                logger.info("🔄 종료 처리 중...")
                logger.info("=" * 60)
                
                # 로그아웃
                perform_logout(driver)
                logger.info("✅ 로그아웃 완료")
                
                # 쿠키 저장
                if mobile:
                    save_cookies(driver, mobile)
                    logger.info(f"✅ {mobile} 쿠키 저장 완료")
                
                # 브라우저 종료
                driver.quit()
                logger.info("✅ 브라우저 종료 완료")
                
                # 최종 메시지
                close_time = time.strftime('%p %I시%M분%S초', time.localtime())
                logger.info("=" * 60)
                logger.info(f"🎉 {mobile} 프로젝트가 {close_time}에 완료되었습니다")
                logger.info("=" * 60)
                
            except Exception as e:
                logger.error(f"❌ 종료 처리 중 예외 발생: {e}", exc_info=True)
        else:
            logger.warning("⚠️  driver가 초기화되지 않았습니다")


if __name__ == "__main__":
    main()


In [18]:
# ===== 셀 4: 반복 실행 함수 (테스트 전용) =====

def test_roof_bands_repeat(duration_hours=0.1, sleep_range=(2, 5)):
    """
    roof_bands 반복 테스트 함수
    
    Args:
        duration_hours: 실행 시간 (기본: 0.1시간 = 6분)
        sleep_range: 대기 시간 범위 (기본: 2~5초)
    """
    
    e_time = time.time() + 60 * 60 * duration_hours
    
    start_time = time.strftime('%p %I시%M분%S초', time.localtime())
    end_time = time.strftime('%p %I시%M분%S초', time.localtime(e_time))
    
    logger.info("=" * 60)
    logger.info(f"🧪 테스트 시작 ({duration_hours}시간)")
    logger.info(f"🕐 {start_time} ~ {end_time}")
    logger.info("=" * 60)
    
    rotation_count = 0
    
    while time.time() < e_time:
        rotation_count += 1
        logger.info(f"🔄 {rotation_count}회차 시작")
        
        failed = roof_bands(driver, xpath_dict, BAND_LIST, TXT_DIR, IMAGE_DIR, mobile_num=mobile)
        
        logger.info(f"✅ {rotation_count}회차 완료 (실패: {len(failed)}개)")
        
        if time.time() < e_time:
            rand_sleep = random.randint(*sleep_range)
            logger.info(f"😴 {rand_sleep}초 휴식...")
            time.sleep(rand_sleep)
    
    logger.info(f"🎉 테스트 완료! (총 {rotation_count}회)")

# ✅ 테스트 실행! (6분)
test_roof_bands_repeat(duration_hours=0.1, sleep_range=(2, 5))

# ✅ 또는 30초 테스트
# test_roof_bands_repeat(duration_hours=0.0083, sleep_range=(1, 2))


[01:54:52] [INFO] ============================================================
[01:54:52] [INFO] 🧪 테스트 시작 (0.1시간)
[01:54:52] [INFO] 🕐 AM 01시54분52초 ~ AM 02시00분52초
[01:54:52] [INFO] ============================================================
[01:54:52] [INFO] 🔄 1회차 시작
[01:54:52] [INFO] [01027851965] 🚀 밴드 순회 시작 (총 9개)
[01:54:52] [INFO] [01027851965] [1/9] 밴드 처리 시작
[01:55:03] [WARNING] 클릭 실패, 재시도: //*[@id="content"]/section/div[2]/div/ul/li[3]/div/div/a/div[1]/div/span
[01:55:14] [WARNING] [01027851965] [1] ⚠️  에러 (TimeoutException): Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xa158c3
	0xa15904
	0x82
[01:55:14] [ERROR] Alert/confirm 창 제어 실패: Message: no such alert
  (Session info: chrome=142.0.7444.60)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xa158c3
	0xa15904
	0x82e5bb
	0x820ecc
	0x85189b
	0x89ca04
	0x85120a
	0x89c766
	0x86dac0
	0x86ede4
	0xc97974
	0xc92bea
	0xa3e5b4
	0xa2dd28
	0xa34d8d
	0xa1ded8
	0xa1e09c
	0xa07d1a
	0x76f87ba9
	

In [19]:
perform_logout(driver)
save_cookies(driver)
close_time = time.strftime('%p %I시%M분%S초', time.localtime())
print(f'이 {mobile}project는 {close_time}에 완료하여 logout하였습니다.')
driver.quit()

[02:14:18] [INFO] [INFO] 로그아웃 절차 시작
[02:14:18] [INFO] [human_delay] click 단계: 0.73초 대기
[02:14:19] [INFO] [human_delay] click 단계: 0.99초 대기
[02:14:20] [INFO] [human_delay] click 단계: 0.64초 대기


NameError: name 'save_cookies' is not defined

In [22]:
from src.account_manager import save_cookies
save_cookies(driver)

[03:43:08] [INFO] [INFO] 쿠키 저장 완료: d:\coding\band_project\accounts/01027851965/cookies\cookies.json


In [23]:
driver.quit()